## `get_pStringMatrix(L)`

Given a `LinearDifferentialOperator` `L` where `L.pFunctions` is the array
$$[p_0, p_1, \ldots, p_n],$$
constructs a matrix whose $(i+1)(j+1)$-entry is a string "pij" which denotes the $j$th derivative of $p_{i}$:
$$\begin{bmatrix}\text{p00} & \cdots & \text{p0(n-1)}\\ \vdots & \ddots & \vdots\\ \text{p(n-1)0} & \cdots & \text{p(n-1)(n-1)}\end{bmatrix}.$$

In [ ]:
function get_pStringMatrix(L::LinearDifferentialOperator)
    if isa(L, LinearDifferentialOperator)
        pFunctions = L.pFunctions
    else
        pFunctions = L.symPFunctions
    end
    n = length(pFunctions)-1
    pStringMatrix = Array{String}(n,n)
    for i in 0:(n-1)
        for j in 0:(n-1)
            pStringMatrix[i+1,j+1] = string("p", i,j)
        end
    end
    return pStringMatrix
end

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator whose `pStringMatrix` is to be constructed.

**Returns**
* `get_pStringMatrix`: `Array`
    * Returns a matrix whose $(i+1)(j+1)$-entry is a string "pij".

**Example**

In [ ]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)

# pFunctions = [t->1 t->t+1 t->t^2+t+1]
# L = LinearDifferentialOperator(pFunctions, interval, symL)
L = get_L(symL)

get_pStringMatrix(L)

In [ ]:
# Using get_L
L = get_L(symL)
get_pStringMatrix(L)

## `get_symUvForm(L, u, v; substitute=true)`

Create the symbolic expression for $[uv](t)$, defined as
\begin{align*}
[uv](t) := \sum_{m=1}^n\sum_{j+k=m-1}(-1)^j u^{(k)}(t)(p_{n-m}\bar{v})^{(j)}(t).
\end{align*}

In [ ]:
function get_symUvForm(L::LinearDifferentialOperator, u::SymPy.Sym, v::SymPy.Sym; substitute = true)
    symL = L.symL
    symPFunctions, t = symL.symPFunctions, symL.t
    n = length(symPFunctions)-1
    if substitute
        pFunctionSymbols = symPFunctions
    else
        pFunctionSymbols = [SymFunction(string("p", i))(t) for i in 0:(n-1)]
    end
    sum = 0
    for m = 1:n
        for (j,k) in partition(m-1)
            summand = (-1)^j * get_symDeriv(u, t, k) * get_symDeriv(pFunctionSymbols[n-m+1] * conj(v), t, j)
            sum += summand
        end
    end
    sum = expand(sum)
    return sum
end

**Parameters**
* `L`: `LinearDifferentialOperator`
    * Linear differential operator whose `L.pFunctions` are to become the $p_{n-m}$ terms in $[uv](t)$.
* `u`, `v`: `SymPy.Sym`
    * Symbolic expression of $u(t)$, $v(t)$.
* `substitute*`: `Bool`
    * Boolean indicating whether to substitute the symbolic expression of $p_i$ (`L.pFunctions`[i]) for the generic expression $p_i(t)$ created using `SymFunction("pi")(t)`.

**Returns**
* `get_symUvForm`: `SymPy.Sym`
    * Returns the symbolic expression of $[uv](t)$ defined above, where $u(t)$, $v(t)$ are
        * the generic expressions $u(t)$, $v(t)$ if `substitute = false`, or
        * explicit symbolic expressions if `substitute = true`.

**Example**

In [ ]:
t = symbols("t")
symPFunctions = [1 t+1 t^2+t+1]
interval = (0, 1)
symL = SymLinearDifferentialOperator(symPFunctions, interval, t)

# pFunctions = [t->1 t->t+1 t->t^2+t+1]
# L = LinearDifferentialOperator(pFunctions, interval, symL)
L = get_L(symL)

t = symbols("t")
u, v = SymFunction("u")(t), SymFunction("v")(t) # u, v as generic functions of t
get_symUvForm(L, u, v; substitute = false)

In [ ]:
get_symUvForm(L, u, v; substitute = true)